In [1]:
from cmsis_svd.parser import SVDParser
import json
import serial
parser = SVDParser.for_packaged_svd("STMicro", "STM32F446x.svd")
dev = parser.get_device()
peripherals = dev.to_dict()['peripherals']

def reshape_dict(register, periphery):
    register['fields'] = {field['name']:field for field in register['fields']}
    register['address'] = periphery['base_address'] + register['address_offset']
    return register

res = {}
for per in peripherals:
    res[per['name']] = {r['name']:reshape_dict(r, per) for r in per['registers']}

In [2]:
TIM_FREQ = 80_000_000
desired_freq = 5000
arr = (TIM_FREQ // (2 * desired_freq)) - 1

In [12]:
import zlib

ser = serial.Serial("/dev/ttyACM3", baudrate=1000000, timeout=5)

start = b'\x11'
end = b'\xa0'
def pack(data):
    crc = zlib.crc32(data).to_bytes(4, byteorder="little")
    return start + data + crc + end

In [5]:
FORCE_LOW = 4
TOGGLE = 3

In [90]:
gpioa = Periph("GPIOA", res)

In [97]:
pg = ProgramGen()

tim2 = Timer('TIM2', res)
tim1 = Timer('TIM1', res)


tim2.set_autoreload(arr)
tim1.set_autoreload(arr)

tim2.set_counter(0)
tim1.set_counter(0)

tim2.toggle_channel(3, 1)
tim1.toggle_channel(3, 1)

tim2.CCMR2_Output.set_field(tim2.CCMR2_Output.OC3M, TOGGLE)
tim1.CCMR2_Output.set_field(tim1.CCMR2_Output.OC3M, TOGGLE)


pg.add_frame([tim2, tim1, gpioa], 90)

gpioa.ODR.set_bit(4)
tim1.start()
tim2.start()


pg.add_frame([tim2, tim1, gpioa], 100)

tim2.stop()
tim1.stop()

tim2.CCMR2_Output.set_field(tim2.CCMR2_Output.OC3M, FORCE_LOW)
tim1.CCMR2_Output.set_field(tim1.CCMR2_Output.OC3M, FORCE_LOW)

pg.add_frame([tim2, tim1, gpioa], 350)

gpioa.ODR.reset_bit(4)

pg.add_frame([tim2, tim1, gpioa], 400)


prg_bin = pg.generate_program()

In [98]:
pg.print_program()

@ 90
    TIM2->CCMR2_Output:0x4000001c:0x30
    TIM2->CCER     :0x40000020:0x100
    TIM2->CNT      :0x40000024:0x0
    TIM2->ARR      :0x4000002c:0x1f3f
    TIM1->CCMR2_Output:0x4001001c:0x30
    TIM1->CCER     :0x40010020:0x100
    TIM1->CNT      :0x40010024:0x0
    TIM1->ARR      :0x4001002c:0x1f3f
@ 100
    TIM2->CR1      :0x40000000:0x1
    TIM2->CCMR2_Output:0x4000001c:0x30
    TIM2->CNT      :0x40000024:0x0
    TIM1->CR1      :0x40010000:0x1
    TIM1->CCMR2_Output:0x4001001c:0x30
    TIM1->CNT      :0x40010024:0x0
    GPIOA->ODR     :0x40020014:0x10
@ 350
    TIM2->CR1      :0x40000000:0x0
    TIM2->CCMR2_Output:0x4000001c:0x40
    TIM2->CNT      :0x40000024:0x0
    TIM1->CR1      :0x40010000:0x0
    TIM1->CCMR2_Output:0x4001001c:0x40
    TIM1->CNT      :0x40010024:0x0
@ 400
    TIM2->CCMR2_Output:0x4000001c:0x40
    TIM2->CNT      :0x40000024:0x0
    TIM1->CCMR2_Output:0x4001001c:0x40
    TIM1->CNT      :0x40010024:0x0
    GPIOA->ODR     :0x40020014:0x0


In [100]:
ser.read_all(); ser.write(pack(prg_bin)); print(ser.read_until()); ser.write(b"\xc6")

b'Program successfully received!\r\n'


1

In [11]:
ser.close()

In [7]:
import json


class Register(object):
    '''
    Class, holding description for low-level C-like registers,
    described by desc (generated from SVD file)

        main usage:
        1. create instance
        gpioa = Register(hw_desc['GPIOA']['ODR'])
        2. use set_field to set bit values. This function supports
        both setting and resetting, i.e.
        gpioa.set_field(gpioa.ODR10, 1)
        gpioa.set_field(gpioa.ODR10, 0)

    '''

    def __init__(self, desc):
        self.desc = desc
        for key in desc:
            setattr(self, key, desc[key])

        for field in desc['fields']:
            setattr(self, field, desc['fields'][field])

        self._value = self.reset_value
        self._prev_value = self.reset_value

    @property
    def value(self):
        return self._value

    @value.setter
    def value(self, val):
        if val > (2**self.size) - 1:
            raise ValueError(
                f"Cant set val {val} to register having {self.size} bits")
        self._value = val

    def __repr__(self):
        tmp = dict(self.desc)
        del tmp['fields']
        c = "Instance of class Register:\n"
        return c + "{" + "\n".join("{!r}: {!r},"
                                   .format(k, v) for k, v in tmp.items()) + "}"

    def validate(self, offset, bit_width):
        assert offset + bit_width < self.size, \
            "Trying to set bits off register"
        assert "write" in self.access, \
            "Trying to modify read-only register"
        assert ((((2**bit_width) - 1) << offset) & (~self.reset_mask)) == 0, \
            "Trying to set bits on read-only positions"

    def set_bit(self, offset):
        self.validate(offset, 1)
        self.value = self.value | (1 << offset)

    def reset_bit(self, offset):
        self.validate(offset, 1)
        self.value = self.value & ~(1 << offset)

    def set_bits(self, offset, bit_width, value):
        self.validate(offset, bit_width)
        self.value = self.value | (value << offset)

    def reset_bits(self, offset, bit_width):
        self.validate(offset, bit_width)
        self.value = self.value & ~(((2**bit_width) - 1) << offset)

    def set_field(self, field, value):
        offset = field['bit_offset']
        width = field['bit_width']
        self.reset_bits(offset, width)
        self.set_bits(offset, width, value)

    def reset_value(self):
        self.value = self.reset_value


class Periph(object):
    """Base class for uC peripherals"""

    def __init__(self, name, hw_desc):
        super(Periph, self).__init__()
        # with open(hw_desc, "r") as f:
        #    self.hw_desc = json.load(f)
        self.hw_desc = hw_desc
        self.name = name
        self.registers = hw_desc[name]
        regs = {}
        for regname, regdata in self.registers.items():
            reg = Register(regdata)
            setattr(self, regname, reg)
            regs[regname] = reg
        self.registers = regs

    def get_modified_regs(self, update=True):
        '''returns list of changed registers from reset (if update=False)
            or from previous call (if update=True)
          registers with prev_value == -1 are always updated,
          which is useful for dynamic registers (e.g. cnter)
        '''
        regs = [reg for reg in self.registers.values()
                if reg.value != reg._prev_value]
        if update:
            for r in regs:
                if r._prev_value >= 0:
                    r._prev_value = r.value
        return regs


class Timer(Periph):
    """docstring for Timer"""

    def __init__(self, *args, **kwargs):
        super(Timer, self).__init__(*args, **kwargs)
        # do not update prev_value for CNT,
        # instead set it each time its requested
        self.CNT._prev_value = -1
        self.CCMR2_Output._prev_value = -1

    def toggle_channel(self, channel, value):
        """
        must set TIM->CCER->CC1E to enable first channel, and so on
        """
        try:
            self.CCER.set_field(self.CCER.fields[f'CC{channel}E'], value)
        except KeyError:
            print(f"Timer does not have this channel: {channel}")

    def toggle_polarity(self, channel, value):
        '''
        bit TIM->CCER->CC1P controls polarity of first channel
        value 0 -> active high
        value 1 -> active low'''
        try:
            self.CCER.set_field(self.CCER.fields[f'CC{channel}P'], value)
        except KeyError:
            raise ValueError(f"Timer does not have this channel: {channel}")

    def set_autoreload(self, value):
        '''
        value at which counter will
        automaticlly reload and generate GPIO toggle
        TIM->ARR'''
        self.ARR.value = value

    def set_counter(self, value):
        # TIM->CNT - current value of a counter
        self.CNT.value = value

    def start(self):
        # need to set TIM->CR1->CEN to enable counter
        self.CR1.set_field(self.CR1.CEN, 1)

    def stop(self):
        # need to reset TIM->CR1->CEN to disable counter
        self.CR1.set_field(self.CR1.CEN, 0)


class ProgramGen(object):
    """docstring for ProgramGen"""

    def __init__(self):
        super(ProgramGen, self).__init__()
        self.frames = []

        # end of frame
        self.eof = 0xABBCCDDE
        # end of programm
        self.eop = 0xFFFFFFFF

    @staticmethod
    def _flatten_cmds(cmds):
        return [elem for iterable in cmds for elem in iterable[1:]]

    @staticmethod
    def _binarize(vals):
        data_bin = b"".join([item.to_bytes(4, byteorder="little")
                             for item in vals])
        return data_bin

    def print_program(self):
        for frame in self.frames:
            print(f"@ {frame['time']}")
            for name, addr, val in frame['cmds']:
                print(f"    {name:<15}:{hex(addr)}:{hex(val)}")

    def add_frame(self, hws, time):
        cmds = []
        for hw in hws:
            regs = hw.get_modified_regs()
            cmds.extend([(f"{hw.name}->{r.name}", r.address, r.value)
                         for r in regs])

        self.frames.append(dict(time=time,
                                cmds=cmds))

    def generate_program(self):
        prg = []
        for frame in self.frames:
            prg.append(frame['time'])
            prg.extend(self._flatten_cmds(frame['cmds']))
            prg.append(self.eof)
        prg.append(self.eop)
        return self._binarize(prg)


In [14]:
with open("hw_desc.json", "w") as f:
    json.dump(res, f, indent=4)